TODO
- Import
- Read data (vocab, sentences)

In [1]:
use_cuda = True
batch_size = 1

# Import library

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

import time
import math
import random
import unicodedata
import string
import re
import scripts.text

# Load data

In [3]:
data_path = './processed-data/id.1000/'
en_vocab_path = data_path + 'train.10k.en.vocab'
de_vocab_path = data_path + 'train.10k.de.vocab'

In [4]:
en_words, en_vocab, _ = scripts.text.load_vocab(en_vocab_path)
de_words, de_vocab, _ = scripts.text.load_vocab(de_vocab_path)

# Loading vocab file ./processed-data/id.1000/train.10k.en.vocab ...
  num words = 1000
# Loading vocab file ./processed-data/id.1000/train.10k.de.vocab ...
  num words = 1000


In [5]:
# Read train data
en_train_sentences = []
with open(data_path + 'train.10k.en', 'r') as f:
    for line in f:
        en_train_sentences.append(line.split())
        
de_train_sentences = []
with open(data_path + 'train.10k.de', 'r') as f:
    for line in f:
        de_train_sentences.append(line.split())

In [6]:
# Read validation data
en_valid_sentences = []
with open(data_path + 'valid.100.en', 'r') as f:
    for line in f:
        en_valid_sentences.append(line.split())
        
de_valid_sentences = []
with open(data_path + 'valid.100.de', 'r') as f:
    for line in f:
        de_valid_sentences.append(line.split())

# Build model

## Using RNNs

In [7]:
class EncoderRNN(nn.Module):
    """
        Model's encoder using RNN.
    """

    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)

    def forward(self, input_sentence, hidden):
        sentence_len = len(input_sentence)
        embedded = self.embedding(input_sentence).view(sentence_len, batch_size, -1)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden
    
    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))
        if use_cuda:
            hidden = hidden.cuda()
        return hidden

In [8]:
class DecoderRNN(nn.Module):
    """
        Model's decoder using RNN.
    """

    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super(DecoderRNN, self).__init__()

        self.embedding = nn.Embedding(output_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_vector, hidden):
        output = self.embedding(input_vector).view(1, batch_size, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output = self.log_softmax(self.out(output[0]))
        return output, hidden
    
    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))
        if use_cuda:
            hidden = hidden.cuda()
        return hidden

# Training model

## Using RNN

In [9]:
teacher_forcing_ratio = 0.5